# Neural Network - Binary Classification

In [18]:
import pandas as pd
data = pd.read_csv('../../../datasets/breast_cancer_data.csv')
target = pd.read_csv('../../../datasets/breast_cancer_target.csv')

In [19]:
data.head()

,radius_mean,texture_mean,perimeter_mean,area_mean,smoothness_mean,compactness_mean,concavity_mean,concave_points_mean,symmetry_mean,fractal_dimension_mean,...,radius_worst,texture_worst,perimeter_worst,area_worst,smoothness_worst,compactness_worst,concavity_worst,concave_points_worst,symmetry_worst,fractal_dimension_worst
0,17.99,10.38,122.80,1001.0,0.11840,0.27760,0.3001,0.14710,0.2419,0.07871,...,25.38,17.33,184.60,2019.0,0.1622,0.6656,0.7119,0.2654,0.4601,0.11890
1,20.57,17.77,132.90,1326.0,0.08474,0.07864,0.0869,0.07017,0.1812,0.05667,...,24.99,23.41,158.80,1956.0,0.1238,0.1866,0.2416,186.0000,275.0000,0.08902
2,19.69,21.25,130.00,1203.0,0.10960,0.15990,0.1974,0.12790,0.2069,0.05999,...,23.57,25.53,152.50,1709.0,0.1444,0.4245,0.4504,243.0000,0.3613,0.08758
3,11.42,20.38,77.58,386.1,0.14250,0.28390,0.2414,0.10520,0.2597,0.09744,...,14.91,26.50,98.87,567.7,0.2098,0.8663,0.6869,0.2575,0.6638,173.00000
4,20.29,14.34,135.10,1297.0,0.10030,0.13280,198.0000,0.10430,0.1809,0.05883,...,22.54,16.67,152.20,1575.0,0.1374,205.0000,0.4000,0.1625,0.2364,0.07678


In [22]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.25)

In [23]:
import keras
from keras.models import Sequential
from keras.layers import Dense

In [44]:
# units = Quantos neurônios farão parte da camada oculta
# Para determinar o número de neurônios: (número de atributos previsores + número de classes)/ 2
# activation: função de ativação
# kernel_initializer: Incialização dos pesos
# input_dim = Quantos elementos na camada de entrada

classifier = Sequential()
# Camada oculta
classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
# Camada oculta (o "input_dim" deve ser colocado apenas na primeira camada oculta)
classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))


# Camada de saída 
# Usamos a sigmoid, pois é um problema de classificação binária
classifier.add(Dense(units=1, activation='sigmoid'))

# Learnig Rate
# Decay: Quanto o learning rate será decrementado a cada iteração
# clipvalue: Uma forma de "prender" o valor para que não fique muito disperso no gradiente
my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)

# optimizer: Função para ajuste dos pesos (descida do gradiente...)
# loss: Função de perda. Como o erro será calculado
# metrics: Qual a métrica que será utilizada para avaliação
# classifier.compile(optimizer='adam', loss='binary_crossentropy', metrics=['binary_accuracy'])

classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])



# Calcular o erro para "batch_size" e depois atualizar os pesos
# epochs: Quantas épocas eu quero executar o treinamento
classifier.fit(X_train, y_train, batch_size = 10, epochs=100)

weights0 = classifier.layers[0].get_weights()
weights1 = classifier.layers[1].get_weights()

Epoch 1/100
426/426 [==============================] - 0s 1ms/step - loss: 0.9777 - binary_accuracy: 0.6338
Epoch 2/100
426/426 [==============================] - 0s 165us/step - loss: 0.5043 - binary_accuracy: 0.7300
Epoch 3/100
426/426 [==============================] - 0s 186us/step - loss: 0.4919 - binary_accuracy: 0.7277
Epoch 4/100
426/426 [==============================] - 0s 131us/step - loss: 0.4632 - binary_accuracy: 0.7512
Epoch 5/100
426/426 [==============================] - 0s 136us/step - loss: 0.4590 - binary_accuracy: 0.7418
Epoch 6/100
426/426 [==============================] - 0s 146us/step - loss: 0.4356 - binary_accuracy: 0.7981
Epoch 7/100
426/426 [==============================] - 0s 125us/step - loss: 0.3848 - binary_accuracy: 0.8169
Epoch 8/100
426/426 [==============================] - 0s 154us/step - loss: 0.4266 - binary_accuracy: 0.8192
Epoch 9/100
426/426 [==============================] - 0s 152us/step - loss: 0.4057 - binary_accuracy: 0.8474
Epoch 10/100

426/426 [==============================] - 0s 167us/step - loss: 0.3079 - binary_accuracy: 0.8967
Epoch 75/100
426/426 [==============================] - 0s 140us/step - loss: 0.3791 - binary_accuracy: 0.8897
Epoch 76/100
426/426 [==============================] - 0s 131us/step - loss: 0.3341 - binary_accuracy: 0.8967
Epoch 77/100
426/426 [==============================] - 0s 142us/step - loss: 0.7815 - binary_accuracy: 0.8568
Epoch 78/100
426/426 [==============================] - 0s 132us/step - loss: 0.5043 - binary_accuracy: 0.8826
Epoch 79/100
426/426 [==============================] - 0s 142us/step - loss: 0.9892 - binary_accuracy: 0.8474
Epoch 80/100
426/426 [==============================] - 0s 129us/step - loss: 0.8796 - binary_accuracy: 0.8638
Epoch 81/100
426/426 [==============================] - 0s 139us/step - loss: 0.7861 - binary_accuracy: 0.8662
Epoch 82/100
426/426 [==============================] - 0s 137us/step - loss: 0.8735 - binary_accuracy: 0.8615
Epoch 83/100
4

In [40]:
predictions = classifier.predict(X_test)
predictions = (predictions > 0.5)

In [41]:
from sklearn.metrics import confusion_matrix, accuracy_score
precision = accuracy_score(y_test, predictions)
print(precision)

precision = confusion_matrix(y_test, predictions)
print(precision)


0.818181818182
[[45 11]
 [15 72]]


In [43]:
result = classifier.evaluate(X_test, y_test)
result

143/143 [==============================] - 0s 83us/step


[1.5097135263723094, 0.818181819849081]

## Cross Validation

In [51]:
from keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
import keras

In [52]:
def build_network():
    classifier = Sequential()
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    classifier.add(Dense(units=1, activation='sigmoid'))
    my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)
    classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return classifier


In [53]:
classifier = KerasClassifier(build_fn=build_network, epochs=100, batch_size=10)

In [54]:
results = cross_val_score(estimator=classifier, X=X_train, y=y_train, cv=10, scoring='accuracy')

Epoch 1/100
383/383 [==============================] - 1s 2ms/step - loss: 1.0968 - binary_accuracy: 0.6084
Epoch 2/100
383/383 [==============================] - 0s 302us/step - loss: 0.4975 - binary_accuracy: 0.6632
Epoch 3/100
383/383 [==============================] - 0s 241us/step - loss: 0.4586 - binary_accuracy: 0.7572
Epoch 4/100
383/383 [==============================] - 0s 166us/step - loss: 0.4370 - binary_accuracy: 0.7885
Epoch 5/100
383/383 [==============================] - 0s 158us/step - loss: 0.4710 - binary_accuracy: 0.8198
Epoch 6/100
383/383 [==============================] - 0s 134us/step - loss: 0.3879 - binary_accuracy: 0.8198
Epoch 7/100
383/383 [==============================] - 0s 138us/step - loss: 0.4299 - binary_accuracy: 0.8355
Epoch 8/100
383/383 [==============================] - 0s 167us/step - loss: 0.3817 - binary_accuracy: 0.8486
Epoch 9/100
383/383 [==============================] - 0s 176us/step - loss: 0.3834 - binary_accuracy: 0.8198
Epoch 10/100

383/383 [==============================] - 0s 230us/step - loss: 0.4827 - binary_accuracy: 0.8642
Epoch 75/100
383/383 [==============================] - 0s 201us/step - loss: 0.3735 - binary_accuracy: 0.8642
Epoch 76/100
383/383 [==============================] - 0s 216us/step - loss: 0.3950 - binary_accuracy: 0.8773
Epoch 77/100
383/383 [==============================] - 0s 193us/step - loss: 0.3353 - binary_accuracy: 0.8930
Epoch 78/100
383/383 [==============================] - 0s 245us/step - loss: 0.4315 - binary_accuracy: 0.8668
Epoch 79/100
383/383 [==============================] - 0s 199us/step - loss: 0.3843 - binary_accuracy: 0.8747
Epoch 80/100
383/383 [==============================] - 0s 230us/step - loss: 0.3528 - binary_accuracy: 0.8851
Epoch 81/100
383/383 [==============================] - 0s 279us/step - loss: 0.3880 - binary_accuracy: 0.8773
Epoch 82/100
383/383 [==============================] - 0s 287us/step - loss: 0.3505 - binary_accuracy: 0.8825
Epoch 83/100
3

383/383 [==============================] - 0s 267us/step - loss: 1.0145 - binary_accuracy: 0.8146
Epoch 49/100
383/383 [==============================] - 0s 342us/step - loss: 1.1487 - binary_accuracy: 0.7990
Epoch 50/100
383/383 [==============================] - 0s 313us/step - loss: 1.0443 - binary_accuracy: 0.8120
Epoch 51/100
383/383 [==============================] - 0s 309us/step - loss: 1.2513 - binary_accuracy: 0.7807
Epoch 52/100
383/383 [==============================] - 0s 300us/step - loss: 1.0254 - binary_accuracy: 0.8329
Epoch 53/100
383/383 [==============================] - 0s 328us/step - loss: 1.2147 - binary_accuracy: 0.8146
Epoch 54/100
383/383 [==============================] - 0s 391us/step - loss: 1.3122 - binary_accuracy: 0.7755
Epoch 55/100
383/383 [==============================] - 0s 247us/step - loss: 1.0245 - binary_accuracy: 0.8146
Epoch 56/100
383/383 [==============================] - 0s 339us/step - loss: 0.9799 - binary_accuracy: 0.8460
Epoch 57/100
3

383/383 [==============================] - 0s 352us/step - loss: 0.6190 - binary_accuracy: 0.8120
Epoch 23/100
383/383 [==============================] - 0s 212us/step - loss: 0.3863 - binary_accuracy: 0.8564
Epoch 24/100
383/383 [==============================] - 0s 464us/step - loss: 0.4235 - binary_accuracy: 0.8721
Epoch 25/100
383/383 [==============================] - 0s 522us/step - loss: 0.5022 - binary_accuracy: 0.8512
Epoch 26/100
383/383 [==============================] - 0s 414us/step - loss: 0.4815 - binary_accuracy: 0.8486
Epoch 27/100
383/383 [==============================] - 0s 429us/step - loss: 0.5011 - binary_accuracy: 0.8198
Epoch 28/100
383/383 [==============================] - 0s 404us/step - loss: 0.6287 - binary_accuracy: 0.8564
Epoch 29/100
383/383 [==============================] - 0s 365us/step - loss: 0.6578 - binary_accuracy: 0.8877
Epoch 30/100
383/383 [==============================] - 0s 406us/step - loss: 0.7958 - binary_accuracy: 0.8277
Epoch 31/100
3

383/383 [==============================] - 0s 184us/step - loss: 1.2910 - binary_accuracy: 0.8433
Epoch 96/100
383/383 [==============================] - 0s 229us/step - loss: 1.1330 - binary_accuracy: 0.8407
Epoch 97/100
383/383 [==============================] - 0s 204us/step - loss: 1.0332 - binary_accuracy: 0.8329
Epoch 98/100
383/383 [==============================] - 0s 235us/step - loss: 1.0807 - binary_accuracy: 0.8407
Epoch 99/100
383/383 [==============================] - 0s 210us/step - loss: 1.0415 - binary_accuracy: 0.8512
Epoch 100/100
383/383 [==============================] - 0s 224us/step - loss: 1.1357 - binary_accuracy: 0.8251
Epoch 1/100
383/383 [==============================] - 1s 2ms/step - loss: 0.7888 - binary_accuracy: 0.6057
Epoch 2/100
383/383 [==============================] - 0s 345us/step - loss: 0.5074 - binary_accuracy: 0.7520
Epoch 3/100
383/383 [==============================] - 0s 371us/step - loss: 0.5364 - binary_accuracy: 0.7546
Epoch 4/100
383/38

...

384/384 [==============================] - 0s 128us/step - loss: 0.4451 - binary_accuracy: 0.8776
Epoch 63/100
384/384 [==============================] - 0s 144us/step - loss: 0.4193 - binary_accuracy: 0.8724
Epoch 64/100
384/384 [==============================] - 0s 139us/step - loss: 0.4149 - binary_accuracy: 0.8620
Epoch 65/100
384/384 [==============================] - 0s 186us/step - loss: 0.3757 - binary_accuracy: 0.8854
Epoch 66/100
384/384 [==============================] - 0s 166us/step - loss: 0.3912 - binary_accuracy: 0.8776
Epoch 67/100
384/384 [==============================] - 0s 144us/step - loss: 0.4417 - binary_accuracy: 0.8750
Epoch 68/100
384/384 [==============================] - 0s 214us/step - loss: 0.4431 - binary_accuracy: 0.8542
Epoch 69/100
384/384 [==============================] - 0s 111us/step - loss: 0.3989 - binary_accuracy: 0.8958
Epoch 70/100
384/384 [==============================] - 0s 187us/step - loss: 0.6410 - binary_accuracy: 0.8307
Epoch 71/100
3

384/384 [==============================] - 0s 124us/step - loss: 0.3058 - binary_accuracy: 0.9115
Epoch 37/100
384/384 [==============================] - 0s 132us/step - loss: 0.4896 - binary_accuracy: 0.8854
Epoch 38/100
384/384 [==============================] - 0s 129us/step - loss: 0.3004 - binary_accuracy: 0.9010
Epoch 39/100
384/384 [==============================] - 0s 233us/step - loss: 0.7288 - binary_accuracy: 0.8542
Epoch 40/100
384/384 [==============================] - 0s 201us/step - loss: 0.8235 - binary_accuracy: 0.8906
Epoch 41/100
384/384 [==============================] - 0s 108us/step - loss: 0.8491 - binary_accuracy: 0.8776
Epoch 42/100
384/384 [==============================] - 0s 133us/step - loss: 0.8444 - binary_accuracy: 0.8698
Epoch 43/100
384/384 [==============================] - 0s 134us/step - loss: 0.7725 - binary_accuracy: 0.8724
Epoch 44/100
384/384 [==============================] - 0s 158us/step - loss: 0.8308 - binary_accuracy: 0.8724
Epoch 45/100
3

384/384 [==============================] - 0s 142us/step - loss: 0.4040 - binary_accuracy: 0.8281
Epoch 11/100
384/384 [==============================] - 0s 210us/step - loss: 0.7082 - binary_accuracy: 0.8047
Epoch 12/100
384/384 [==============================] - 0s 181us/step - loss: 0.6209 - binary_accuracy: 0.8099
Epoch 13/100
384/384 [==============================] - 0s 219us/step - loss: 0.5473 - binary_accuracy: 0.8073
Epoch 14/100
384/384 [==============================] - 0s 288us/step - loss: 0.7771 - binary_accuracy: 0.8177
Epoch 15/100
384/384 [==============================] - 0s 169us/step - loss: 0.9410 - binary_accuracy: 0.7760
Epoch 16/100
384/384 [==============================] - 0s 253us/step - loss: 0.7986 - binary_accuracy: 0.8385
Epoch 17/100
384/384 [==============================] - 0s 264us/step - loss: 1.0662 - binary_accuracy: 0.7865
Epoch 18/100
384/384 [==============================] - 0s 239us/step - loss: 0.9538 - binary_accuracy: 0.7786
Epoch 19/100
3

384/384 [==============================] - 0s 216us/step - loss: 0.8936 - binary_accuracy: 0.8620
Epoch 84/100
384/384 [==============================] - 0s 126us/step - loss: 0.8365 - binary_accuracy: 0.8672
Epoch 85/100
384/384 [==============================] - 0s 216us/step - loss: 0.8809 - binary_accuracy: 0.8464
Epoch 86/100
384/384 [==============================] - 0s 195us/step - loss: 0.9347 - binary_accuracy: 0.8464
Epoch 87/100
384/384 [==============================] - 0s 163us/step - loss: 0.8986 - binary_accuracy: 0.8542
Epoch 88/100
384/384 [==============================] - 0s 176us/step - loss: 0.9068 - binary_accuracy: 0.8385
Epoch 89/100
384/384 [==============================] - 0s 229us/step - loss: 0.9960 - binary_accuracy: 0.8229
Epoch 90/100
384/384 [==============================] - 0s 244us/step - loss: 0.8911 - binary_accuracy: 0.8516
Epoch 91/100
384/384 [==============================] - 0s 343us/step - loss: 0.9118 - binary_accuracy: 0.8464
Epoch 92/100
3

384/384 [==============================] - 0s 118us/step - loss: 0.5445 - binary_accuracy: 0.8646
Epoch 58/100
384/384 [==============================] - 0s 119us/step - loss: 0.3247 - binary_accuracy: 0.8880
Epoch 59/100
384/384 [==============================] - 0s 108us/step - loss: 0.4940 - binary_accuracy: 0.8359
Epoch 60/100
384/384 [==============================] - 0s 112us/step - loss: 0.5975 - binary_accuracy: 0.8203
Epoch 61/100
384/384 [==============================] - 0s 108us/step - loss: 0.4016 - binary_accuracy: 0.8568
Epoch 62/100
384/384 [==============================] - 0s 111us/step - loss: 0.5980 - binary_accuracy: 0.8464
Epoch 63/100
384/384 [==============================] - 0s 112us/step - loss: 0.4822 - binary_accuracy: 0.8333
Epoch 64/100
384/384 [==============================] - 0s 109us/step - loss: 0.4543 - binary_accuracy: 0.8620
Epoch 65/100
384/384 [==============================] - 0s 106us/step - loss: 0.3600 - binary_accuracy: 0.8724
Epoch 66/100
3

In [58]:
results.mean()
results.std()

0.085469201293840655

# Dropout

Zera alguns neurônios das camadas para reduzir o overffiting


In [61]:
from keras.layers import Dropout

def build_network_with_dropout():
    classifier = Sequential()
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform', input_dim=30))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=16, activation='relu', kernel_initializer='random_uniform'))
    classifier.add(Dropout(0.2))
    classifier.add(Dense(units=1, activation='sigmoid'))
    my_optimizer = keras.optimizers.Adam(lr = 0.001, decay=0.0001, clipvalue=0.5)
    classifier.compile(optimizer=my_optimizer, loss='binary_crossentropy', metrics=['binary_accuracy'])
    
    return classifier